In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from matplotlib import pyplot as plt

In [9]:
all_classes = ['dog', 'horse', 'elephant', 'butterfly', 'chicken', 'cat', 'cow', 'sheep', 'squirrel', 'spider']
folder_names = ['cane', 'cavallo', 'elefante', 'farfalla','gallina', 'gatto', 'mucca', 'pecora', 'scoiattolo', 'ragno']
X = []
Y = []
for i in range(10):
    path = 'C://Users//ADMIN//Desktop//ANIMAL//raw-img//' + folder_names[i]   
    for file in os.listdir(path):
        try: 
            arr_img = cv2.imread(os.path.join(path, file))
            
#             resizing the images
            arr_img = cv2.resize(arr_img, (100,100))
    
            X.append(arr_img)
            Y.append(i)
        except:
            pass
        
X = np.array(X)
Y = np.array(Y)
# Training and Testing set creation + shuffling them
train_X_set, test_X_set, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 100)

In [3]:
# normalizing the data to remove the duplicate data
train_X_set = train_X_set / 255.0
test_X_set = test_X_set / 255.0
total_classes = []
for i in range(10):
    total_classes.append(np.count_nonzero(Y==i))

largest_classes = total_classes[0] + total_classes[9]
print('Amount of samples from the two largest classes: ', largest_classes)
print('Ratio to total number of classes: ', largest_classes/sum(total_classes))
total_sum = sum(total_classes)

random_choice = []
for i in range(test_X_set.shape[0]):
    random_choice.append(np.random.choice(np.arange(0, 10), p = [total_classes[0]/total_sum, total_classes[1]/total_sum, total_classes[2]/total_sum, total_classes[3]/total_sum, total_classes[4]/total_sum, total_classes[5]/total_sum, total_classes[6]/total_sum, total_classes[7]/total_sum, total_classes[8]/total_sum, total_classes[9]/total_sum]))
    
print('First 100 selections of random selector: ', random_choice[0:100])

print(classification_report(Y_test, random_choice, target_names = all_classes))

Amount of samples from the two largest classes:  9684
Ratio to total number of classes:  0.3699148172199091
First 100 selections of random selector:  [7, 0, 5, 3, 4, 7, 8, 0, 9, 8, 7, 0, 0, 1, 5, 5, 9, 4, 6, 0, 2, 4, 9, 9, 6, 0, 4, 4, 4, 9, 6, 4, 7, 0, 0, 1, 3, 9, 9, 5, 8, 4, 2, 7, 9, 9, 9, 4, 9, 5, 5, 2, 7, 0, 5, 1, 5, 7, 2, 0, 9, 4, 4, 0, 0, 9, 0, 5, 1, 6, 9, 3, 0, 5, 4, 8, 3, 9, 4, 8, 0, 1, 1, 8, 1, 9, 7, 1, 9, 5, 4, 1, 8, 1, 5, 0, 9, 8, 4, 7]
              precision    recall  f1-score   support

         dog       0.19      0.19      0.19      1221
       horse       0.11      0.11      0.11       671
    elephant       0.05      0.05      0.05       372
   butterfly       0.08      0.07      0.08       554
     chicken       0.12      0.12      0.12       773
         cat       0.08      0.08      0.08       428
         cow       0.07      0.07      0.07       456
       sheep       0.08      0.08      0.08       431
    squirrel       0.07      0.07      0.07       438
      sp

In [4]:
input_shape = train_X_set[0].shape
no_of_classes = 10
def Training_Function(model):
    history = model.fit(
    x = train_X_set, 
    y = Y_train,
    epochs = 15,
    batch_size = 1000,
    validation_split=0.1,
    verbose=1
    )

    history = pd.DataFrame(history.history)
    display(history)
    
def Testing(model):
    predict_val = model.predict(test_X_set)

    predict_val = np.argmax(predict_val, axis = 1)

    print(classification_report(Y_test, predict_val, target_names = all_classes))
    
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

k_keras = keras.Sequential()

k_keras.add(keras.layers.Conv2D(32, kernel_size = 3, activation='relu', input_shape = input_shape))
k_keras.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

k_keras.add(keras.layers.Conv2D(64, kernel_size = 3, activation='relu'))
k_keras.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

k_keras.add(keras.layers.Conv2D(128, kernel_size = 3, activation='relu'))
k_keras.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

k_keras.add(keras.layers.Flatten())

k_keras.add(keras.layers.Dense(256, activation = 'relu'))

k_keras.add(keras.layers.Dense(128, activation = 'relu'))

k_keras.add(keras.layers.Dense(64, activation = 'relu'))

k_keras.add(keras.layers.Dense(units=no_of_classes, activation = 'softmax'))

k_keras.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

k_keras.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 128)      0

In [5]:
#training the model
Training_Function(k_keras)

Epoch 1/15
18/18 [==============================] - 227s 11s/step - loss: 2.2430 - accuracy: 0.1985 - val_loss: 2.1623 - val_accuracy: 0.2072
Epoch 2/15
18/18 [==============================] - 183s 10s/step - loss: 2.0477 - accuracy: 0.2889 - val_loss: 1.9221 - val_accuracy: 0.3371
Epoch 3/15
18/18 [==============================] - 182s 10s/step - loss: 1.8312 - accuracy: 0.3620 - val_loss: 1.7671 - val_accuracy: 0.3819
Epoch 4/15
18/18 [==============================] - 184s 10s/step - loss: 1.6992 - accuracy: 0.4078 - val_loss: 1.6530 - val_accuracy: 0.4241
Epoch 5/15
18/18 [==============================] - 182s 10s/step - loss: 1.5629 - accuracy: 0.4578 - val_loss: 1.5315 - val_accuracy: 0.4613
Epoch 6/15
18/18 [==============================] - 183s 10s/step - loss: 1.4680 - accuracy: 0.4916 - val_loss: 1.4154 - val_accuracy: 0.5081
Epoch 7/15
18/18 [==============================] - 183s 10s/step - loss: 1.3571 - accuracy: 0.5347 - val_loss: 1.3570 - val_accuracy: 0.5418
Epoch 

,loss,accuracy,val_loss,val_accuracy
0,2.242960,0.198472,2.162346,0.207230
1,2.047654,0.288851,1.922076,0.337067
2,1.831221,0.362026,1.767094,0.381874
3,1.699192,0.407753,1.652994,0.424134
4,1.562915,0.457838,1.531521,0.461303
5,1.467961,0.491568,1.415427,0.508147
6,1.357110,0.534692,1.357028,0.541752
7,1.273051,0.566497,1.370636,0.531568
8,1.216588,0.589813,1.285340,0.568228
9,1.142043,0.619242,1.249369,0.587067


In [7]:
#Testing the model
Testing(k_keras)

205/205 [==============================] - 16s 79ms/step
              precision    recall  f1-score   support

         dog       0.62      0.64      0.63      1221
       horse       0.66      0.53      0.59       671
    elephant       0.45      0.65      0.53       372
   butterfly       0.84      0.62      0.72       554
     chicken       0.69      0.69      0.69       773
         cat       0.47      0.32      0.38       428
         cow       0.50      0.53      0.52       456
       sheep       0.41      0.61      0.49       431
    squirrel       0.60      0.39      0.47       438
      spider       0.76      0.83      0.79      1201

    accuracy                           0.62      6545
   macro avg       0.60      0.58      0.58      6545
weighted avg       0.63      0.62      0.62      6545

